# Post-Quantum RPKI Validation Results - Comprehensive JSON Analysis

**Scientific Analysis of Post-Quantum Signature Algorithms in RPKI**

This notebook provides comprehensive analysis of post-quantum RPKI measurements from the JSON output of `validate.py`, including:

- **Repository Metrics**: File counts, sizes, type breakdowns (certificates, ROAs, manifests, CRLs)
- **Signature Verification**: Verification rates, ASN.1 extraction success, timing statistics
- **Performance Analysis**: Validation times, throughput, per-signature verification times
- **Size Analysis**: Signature sizes, public key sizes, overhead calculations
- **Relative Performance**: Comparisons vs ECDSA baseline
- **Statistical Analysis**: Min/max/avg/median metrics for all measurements

**Data Source**: JSON output from `validate.py` (comprehensive nested structure)  
**Author**: Post-Quantum RPKI Research Team  
**Date**: December 2025  
**Dataset**: Real-world RPKI repository measurements (118,068+ objects)


## 1. Setup and Data Loading


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Publication-quality plotting style
try:
    plt.style.use('seaborn-v0_8-paper')
except OSError:
    try:
        plt.style.use('seaborn-paper')
    except OSError:
        plt.style.use('default')

plt.rcParams.update({
    'font.size': 11,
    'font.family': 'serif',
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 16,
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1
})

print("✓ Libraries loaded successfully")


In [ ]:
# Load data - Primary focus on JSON for detailed nested metrics
csv_path = Path("/work/results.csv")
json_path = Path("/work/results.json")

# Try local paths if Docker paths don't exist
if not json_path.exists():
    json_path = Path("results.json")
if not csv_path.exists():
    csv_path = Path("results.csv")

if not json_path.exists():
    raise FileNotFoundError("results.json not found. Please run validate.py first to generate JSON data.")

# Load JSON data (primary source - contains all nested metrics)
with open(json_path, 'r') as f:
    json_data = json.load(f)

metadata = json_data.get('experiment_metadata', {})
results_list = json_data.get('results', [])
validation_errors = json_data.get('validation_errors', [])

# Convert to DataFrame for easier analysis
df = pd.DataFrame(results_list)

# Also load CSV for comparison (flattened structure)
df_csv = None
if csv_path.exists():
    df_csv = pd.read_csv(csv_path)

print(f"✓ Loaded JSON data: {len(results_list)} algorithm results")
print(f"✓ Experiment date: {metadata.get('date', 'Unknown')}")
print(f"✓ Total objects: {metadata.get('total_objects', 'Unknown'):,}")
print(f"✓ ASN.1 extraction available: {metadata.get('asn1_extraction_available', False)}")
print(f"✓ OQS library available: {metadata.get('oqs_available', False)}")
print(f"\nAvailable metrics in JSON:")
print(f"  - File statistics: file_count, total_size_gb, file_type_breakdown")
print(f"  - Signature verification: verified, failed, verification times, signature sizes")
print(f"  - Performance: validation_time_sec, objects_per_second, scan_time_sec")
print(f"  - Object types: certificates, ROAs, manifests, CRLs")


## 2. Experiment Overview


In [ ]:
# Experiment overview
print("=" * 80)
print("  FIRST REAL POST-QUANTUM RPKI MEASUREMENTS (December 2025)")
print("=" * 80)
print(f"\nExperiment Date: {metadata.get('date', 'Unknown')}")
print(f"Total Objects Validated: {metadata.get('total_objects', 'Unknown'):,}")
print(f"Total Algorithms Tested: {len(df)}")
print(f"Successful Validations: {df['validation_success'].sum()}/{len(df)}")
print("=" * 80)


## 3. Summary Table


In [ ]:
# Display summary table with key metrics
summary_cols = ['algorithm', 'algorithm_standardized', 'nist_security_level', 
                'file_count', 'total_size_gb', 'validation_time_min', 'validation_success']

# Extract file type breakdown if available
if 'file_type_breakdown' in df.columns:
    # Create a summary of file types
    file_types_summary = []
    for idx, row in df.iterrows():
        ftb = row.get('file_type_breakdown', {})
        if isinstance(ftb, dict):
            types_str = ', '.join([f"{k}:{v}" for k, v in ftb.items() if v > 0])
            file_types_summary.append(types_str)
        else:
            file_types_summary.append('N/A')
    df['file_types'] = file_types_summary

summary_df = df[summary_cols].copy()
summary_df['validation_success'] = summary_df['validation_success'].map({True: 'PASS', False: 'FAIL'})
summary_df.columns = ['Algorithm', 'Standardized Name', 'NIST Level', 
                     'File Count', 'Size (GB)', 'Time (min)', 'Status']
summary_df


## 4. Relative Performance vs ECDSA Baseline


In [ ]:
# Calculate relative metrics vs baseline
baseline = None
baseline_idx = None
if 'ecdsa-baseline' in df['algorithm'].values:
    baseline_idx = df[df['algorithm'] == 'ecdsa-baseline'].index[0]
    baseline = df.iloc[baseline_idx]

if baseline is not None:
    df['size_overhead'] = ((df['total_size_gb'] / baseline['total_size_gb'] - 1) * 100).round(2)
    df['time_overhead'] = ((df['validation_time_sec'] / baseline['validation_time_sec'] - 1) * 100).round(2)
    
    comparison_df = df[df['algorithm'] != 'ecdsa-baseline'][['algorithm', 'size_overhead', 'time_overhead']].copy()
    comparison_df.columns = ['Algorithm', 'Size Overhead (%)', 'Time Overhead (%)']
    comparison_df['Size Overhead (%)'] = comparison_df['Size Overhead (%)'].apply(lambda x: f"{x:+.1f}%")
    comparison_df['Time Overhead (%)'] = comparison_df['Time Overhead (%)'].apply(lambda x: f"{x:+.1f}%")
    comparison_df
else:
    print("Baseline data not available for comparison")


## 5. Visualizations

### 5.1 Validation Time Comparison


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#2ecc71' if x else '#e74c3c' for x in df['validation_success']]
bars = ax.bar(df['algorithm'], df['validation_time_min'], color=colors, alpha=0.7, edgecolor='black', linewidth=1.2)
ax.set_xlabel('Algorithm', fontweight='bold')
ax.set_ylabel('Validation Time (minutes)', fontweight='bold')
ax.set_title('RPKI Validation Time: Post-Quantum vs Classical', fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_axisbelow(True)

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.1f}', ha='center', va='bottom', fontweight='bold')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


### 5.2 Repository Size Comparison


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#3498db' if x else '#e74c3c' for x in df['validation_success']]
bars = ax.bar(df['algorithm'], df['total_size_gb'], color=colors, alpha=0.7, edgecolor='black', linewidth=1.2)
ax.set_xlabel('Algorithm', fontweight='bold')
ax.set_ylabel('Repository Size (GB)', fontweight='bold')
ax.set_title('RPKI Repository Size: Post-Quantum vs Classical', fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_axisbelow(True)

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}', ha='center', va='bottom', fontweight='bold')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


### 5.3 Relative Performance vs Baseline


In [ ]:
if baseline is not None:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    pq_df = df[df['algorithm'] != 'ecdsa-baseline']
    
    # Size overhead
    colors_size = ['#e67e22' if x >= 0 else '#27ae60' for x in pq_df['size_overhead']]
    bars1 = ax1.bar(pq_df['algorithm'], pq_df['size_overhead'], color=colors_size, alpha=0.7, edgecolor='black', linewidth=1.2)
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax1.set_xlabel('Algorithm', fontweight='bold')
    ax1.set_ylabel('Size Overhead (%)', fontweight='bold')
    ax1.set_title('Repository Size Overhead vs ECDSA Baseline', fontweight='bold')
    ax1.grid(axis='y', alpha=0.3, linestyle='--')
    ax1.set_axisbelow(True)
    
    for bar in bars1:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height, f'{height:+.1f}%',
                ha='center', va='bottom' if height >= 0 else 'top', fontweight='bold')
    
    # Time overhead
    colors_time = ['#e67e22' if x >= 0 else '#27ae60' for x in pq_df['time_overhead']]
    bars2 = ax2.bar(pq_df['algorithm'], pq_df['time_overhead'], color=colors_time, alpha=0.7, edgecolor='black', linewidth=1.2)
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax2.set_xlabel('Algorithm', fontweight='bold')
    ax2.set_ylabel('Time Overhead (%)', fontweight='bold')
    ax2.set_title('Validation Time Overhead vs ECDSA Baseline', fontweight='bold')
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    ax2.set_axisbelow(True)
    
    for bar in bars2:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height, f'{height:+.1f}%',
                ha='center', va='bottom' if height >= 0 else 'top', fontweight='bold')
    
    plt.setp([ax1.xaxis.get_majorticklabels(), ax2.xaxis.get_majorticklabels()], rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("Baseline data not available")


## 6. Daily Delta Analysis (Bandwidth Overhead)


In [ ]:
# Calculate daily bandwidth overhead (2% daily update rate typical for RPKI)
daily_update_rate = 0.02

if baseline is not None:
    baseline_size = baseline['total_size_bytes']
    baseline_daily_mb = (baseline_size * daily_update_rate) / (1024**2)
    
    print(f"Baseline repository size: {baseline_size / (1024**3):.2f} GB")
    print(f"Assumed daily update rate: {daily_update_rate*100:.1f}%")
    print(f"Baseline daily delta: {baseline_daily_mb:.2f} MB/day\n")
    
    delta_data = []
    for _, row in df.iterrows():
        if row['algorithm'] != 'ecdsa-baseline' and row['total_size_bytes'] > 0:
            size_bytes = row['total_size_bytes']
            overhead_bytes = size_bytes - baseline_size
            overhead_percent = (overhead_bytes / baseline_size * 100) if baseline_size > 0 else 0
            daily_delta_mb = ((size_bytes * daily_update_rate) - (baseline_size * daily_update_rate)) / (1024**2)
            
            delta_data.append({
                'Algorithm': row['algorithm'],
                'Size Overhead (%)': f"{overhead_percent:+.1f}%",
                'Daily Delta (MB/day)': f"{daily_delta_mb:+.2f}"
            })
    
    if delta_data:
        delta_df = pd.DataFrame(delta_data)
        delta_df
else:
    print("Baseline data not available")


## 7. Key Findings


In [ ]:
print("=" * 80)
print("KEY FINDINGS")
print("=" * 80)

if baseline is not None:
    for _, row in df.iterrows():
        if row['algorithm'] != 'ecdsa-baseline':
            print(f"\n{row['algorithm'].upper()}:")
            print(f"  • Size overhead: {row['size_overhead']:+.1f}% vs ECDSA")
            print(f"  • Time overhead: {row['time_overhead']:+.1f}% vs ECDSA")
            if row['validation_success']:
                status_msg = row.get('validation_status', 'PASS')
                if 'rpki-client' in str(status_msg).lower():
                    print(f"  • Status: ✓ PASS with real rpki-client validation")
                else:
                    print(f"  • Status: ✓ PASS")

print("\n" + "=" * 80)
print("SCIENTIFIC CONTRIBUTION")
print("=" * 80)
print("\nThis dataset represents the first real-world measurements of")
print("NIST post-quantum signature algorithms (ML-DSA, Falcon) applied")
print("to the global RPKI repository at scale.")
print("=" * 80)


## 8. Detailed Data


In [ ]:
## 8. Detailed Signature Verification Analysis


In [ ]:
# Extract signature verification metrics from nested JSON structure
sig_verification_data = []

for result in results_list:
    algo = result.get('algorithm', 'unknown')
    sig_ver = result.get('signature_verification', {})
    
    if sig_ver and isinstance(sig_ver, dict):
        sig_verification_data.append({
            'algorithm': algo,
            'sampled': sig_ver.get('sampled', 0),
            'verified': sig_ver.get('verified', 0),
            'failed': sig_ver.get('failed', 0),
            'asn1_failures': sig_ver.get('asn1_extraction_failures', 0),
            'verify_time_sec': sig_ver.get('verify_time_sec', 0),
            'time_per_file_ms': sig_ver.get('time_per_file_sec', 0) * 1000,
            'estimated_total_time_sec': sig_ver.get('estimated_total_time_sec', 0),
            'verification_rate_per_sec': sig_ver.get('verification_rate_per_sec', 0),
            'avg_verify_time_ms': sig_ver.get('avg_verify_time_ms', 0),
            'min_verify_time_ms': sig_ver.get('min_verify_time_ms', 0),
            'max_verify_time_ms': sig_ver.get('max_verify_time_ms', 0),
            'median_verify_time_ms': sig_ver.get('median_verify_time_ms', 0),
            'signature_size_avg_bytes': sig_ver.get('signature_size_avg_bytes', 0),
            'signature_size_min_bytes': sig_ver.get('signature_size_min_bytes', 0),
            'signature_size_max_bytes': sig_ver.get('signature_size_max_bytes', 0),
            'expected_signature_size_bytes': sig_ver.get('expected_signature_size_bytes', 0),
            'expected_public_key_size_bytes': sig_ver.get('expected_public_key_size_bytes', 0),
            'verification_success_rate': (sig_ver.get('verified', 0) / sig_ver.get('sampled', 1)) * 100 if sig_ver.get('sampled', 0) > 0 else 0
        })
    else:
        sig_verification_data.append({
            'algorithm': algo,
            'sampled': 0,
            'verified': 0,
            'failed': 0,
            'asn1_failures': 0,
            'verify_time_sec': 0,
            'time_per_file_ms': 0,
            'estimated_total_time_sec': 0,
            'verification_rate_per_sec': 0,
            'avg_verify_time_ms': 0,
            'min_verify_time_ms': 0,
            'max_verify_time_ms': 0,
            'median_verify_time_ms': 0,
            'signature_size_avg_bytes': 0,
            'signature_size_min_bytes': 0,
            'signature_size_max_bytes': 0,
            'expected_signature_size_bytes': 0,
            'expected_public_key_size_bytes': 0,
            'verification_success_rate': 0
        })

sig_df = pd.DataFrame(sig_verification_data)
sig_df = sig_df[sig_df['sampled'] > 0]  # Only show algorithms with signature verification data

if len(sig_df) > 0:
    print("Signature Verification Metrics:")
    display_cols = ['algorithm', 'sampled', 'verified', 'failed', 'asn1_failures', 
                    'verification_success_rate', 'avg_verify_time_ms', 'verification_rate_per_sec']
    display_df = sig_df[display_cols].copy()
    display_df.columns = ['Algorithm', 'Sampled', 'Verified', 'Failed', 'ASN.1 Errors', 
                          'Success Rate (%)', 'Avg Verify Time (ms)', 'Rate (sig/s)']
    display_df['Success Rate (%)'] = display_df['Success Rate (%)'].round(1)
    display_df['Avg Verify Time (ms)'] = display_df['Avg Verify Time (ms)'].round(2)
    display_df['Rate (sig/s)'] = display_df['Rate (sig/s)'].round(1)
    display_df
else:
    print("No signature verification data available")


### 8.1 Signature Verification Timing Distribution


In [ ]:
# Visualize signature verification timing statistics
if len(sig_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Average verification time
    ax1 = axes[0, 0]
    bars1 = ax1.bar(sig_df['algorithm'], sig_df['avg_verify_time_ms'], 
                    color='#3498db', alpha=0.7, edgecolor='black', linewidth=1.2)
    ax1.set_xlabel('Algorithm', fontweight='bold')
    ax1.set_ylabel('Average Verification Time (ms)', fontweight='bold')
    ax1.set_title('Average Signature Verification Time', fontweight='bold')
    ax1.grid(axis='y', alpha=0.3, linestyle='--')
    ax1.set_axisbelow(True)
    for bar in bars1:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height, f'{height:.2f}',
                ha='center', va='bottom', fontweight='bold', fontsize=9)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Verification rate
    ax2 = axes[0, 1]
    bars2 = ax2.bar(sig_df['algorithm'], sig_df['verification_rate_per_sec'],
                    color='#2ecc71', alpha=0.7, edgecolor='black', linewidth=1.2)
    ax2.set_xlabel('Algorithm', fontweight='bold')
    ax2.set_ylabel('Verification Rate (signatures/sec)', fontweight='bold')
    ax2.set_title('Signature Verification Throughput', fontweight='bold')
    ax2.grid(axis='y', alpha=0.3, linestyle='--')
    ax2.set_axisbelow(True)
    for bar in bars2:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height, f'{height:.1f}',
                ha='center', va='bottom', fontweight='bold', fontsize=9)
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Min/Max/Median verification time comparison
    ax3 = axes[1, 0]
    x = np.arange(len(sig_df))
    width = 0.25
    ax3.bar(x - width, sig_df['min_verify_time_ms'], width, label='Min', color='#27ae60', alpha=0.7, edgecolor='black')
    ax3.bar(x, sig_df['avg_verify_time_ms'], width, label='Avg', color='#3498db', alpha=0.7, edgecolor='black')
    ax3.bar(x + width, sig_df['max_verify_time_ms'], width, label='Max', color='#e74c3c', alpha=0.7, edgecolor='black')
    ax3.set_xlabel('Algorithm', fontweight='bold')
    ax3.set_ylabel('Verification Time (ms)', fontweight='bold')
    ax3.set_title('Verification Time Distribution (Min/Avg/Max)', fontweight='bold')
    ax3.set_xticks(x)
    ax3.set_xticklabels(sig_df['algorithm'], rotation=45, ha='right')
    ax3.legend()
    ax3.grid(axis='y', alpha=0.3, linestyle='--')
    ax3.set_axisbelow(True)
    
    # Signature sizes
    ax4 = axes[1, 1]
    x = np.arange(len(sig_df))
    width = 0.35
    ax4.bar(x - width/2, sig_df['signature_size_avg_bytes'], width, 
           label='Actual Avg', color='#9b59b6', alpha=0.7, edgecolor='black')
    ax4.bar(x + width/2, sig_df['expected_signature_size_bytes'], width,
           label='Expected', color='#34495e', alpha=0.7, edgecolor='black')
    ax4.set_xlabel('Algorithm', fontweight='bold')
    ax4.set_ylabel('Signature Size (bytes)', fontweight='bold')
    ax4.set_title('Signature Size: Actual vs Expected', fontweight='bold')
    ax4.set_xticks(x)
    ax4.set_xticklabels(sig_df['algorithm'], rotation=45, ha='right')
    ax4.legend()
    ax4.grid(axis='y', alpha=0.3, linestyle='--')
    ax4.set_axisbelow(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("No signature verification data available for visualization")


## 9. File Type Breakdown Analysis


In [ ]:
# Extract and analyze file type breakdowns
file_type_data = []

for result in results_list:
    algo = result.get('algorithm', 'unknown')
    ftb = result.get('file_type_breakdown', {})
    
    if isinstance(ftb, dict):
        file_type_data.append({
            'algorithm': algo,
            'certificates': ftb.get('certificates', 0),
            'roas': ftb.get('roas', 0),
            'manifests': ftb.get('manifests', 0),
            'crls': ftb.get('crls', 0),
            'other': ftb.get('other', 0),
            'total': result.get('file_count', 0)
        })
    else:
        file_type_data.append({
            'algorithm': algo,
            'certificates': 0,
            'roas': 0,
            'manifests': 0,
            'crls': 0,
            'other': 0,
            'total': result.get('file_count', 0)
        })

ftb_df = pd.DataFrame(file_type_data)

if len(ftb_df) > 0:
    print("File Type Breakdown by Algorithm:")
    display(ftb_df)
    
    # Visualization
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(ftb_df))
    width = 0.15
    
    ax.bar(x - 2*width, ftb_df['certificates'], width, label='Certificates', color='#3498db', alpha=0.7, edgecolor='black')
    ax.bar(x - width, ftb_df['roas'], width, label='ROAs', color='#2ecc71', alpha=0.7, edgecolor='black')
    ax.bar(x, ftb_df['manifests'], width, label='Manifests', color='#e67e22', alpha=0.7, edgecolor='black')
    ax.bar(x + width, ftb_df['crls'], width, label='CRLs', color='#9b59b6', alpha=0.7, edgecolor='black')
    ax.bar(x + 2*width, ftb_df['other'], width, label='Other', color='#95a5a6', alpha=0.7, edgecolor='black')
    
    ax.set_xlabel('Algorithm', fontweight='bold')
    ax.set_ylabel('File Count', fontweight='bold')
    ax.set_title('File Type Distribution by Algorithm', fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(ftb_df['algorithm'], rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)
    plt.tight_layout()
    plt.show()
else:
    print("No file type breakdown data available")


## 10. Object Type Breakdown from Signature Verification


In [ ]:
# Extract object type breakdown from signature verification
object_type_data = []

for result in results_list:
    algo = result.get('algorithm', 'unknown')
    sig_ver = result.get('signature_verification', {})
    
    if sig_ver and isinstance(sig_ver, dict):
        otb = sig_ver.get('object_type_breakdown', {})
        if isinstance(otb, dict):
            object_type_data.append({
                'algorithm': algo,
                'certificate': otb.get('certificate', 0),
                'roa': otb.get('roa', 0),
                'manifest': otb.get('manifest', 0),
                'crl': otb.get('crl', 0),
                'total_sampled': sig_ver.get('sampled', 0)
            })
        else:
            object_type_data.append({
                'algorithm': algo,
                'certificate': 0,
                'roa': 0,
                'manifest': 0,
                'crl': 0,
                'total_sampled': sig_ver.get('sampled', 0)
            })
    else:
        object_type_data.append({
            'algorithm': algo,
            'certificate': 0,
            'roa': 0,
            'manifest': 0,
            'crl': 0,
            'total_sampled': 0
        })

otb_df = pd.DataFrame(object_type_data)
otb_df = otb_df[otb_df['total_sampled'] > 0]  # Only show algorithms with data

if len(otb_df) > 0:
    print("Object Type Breakdown from Signature Verification Sample:")
    display(otb_df)
    
    # Visualization
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(otb_df))
    width = 0.2
    
    ax.bar(x - 1.5*width, otb_df['certificate'], width, label='Certificate', color='#3498db', alpha=0.7, edgecolor='black')
    ax.bar(x - 0.5*width, otb_df['roa'], width, label='ROA', color='#2ecc71', alpha=0.7, edgecolor='black')
    ax.bar(x + 0.5*width, otb_df['manifest'], width, label='Manifest', color='#e67e22', alpha=0.7, edgecolor='black')
    ax.bar(x + 1.5*width, otb_df['crl'], width, label='CRL', color='#9b59b6', alpha=0.7, edgecolor='black')
    
    ax.set_xlabel('Algorithm', fontweight='bold')
    ax.set_ylabel('Object Count (from verification sample)', fontweight='bold')
    ax.set_title('Object Type Distribution in Verification Sample', fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(otb_df['algorithm'], rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)
    plt.tight_layout()
    plt.show()
else:
    print("No object type breakdown data available from signature verification")


## 11. Complete JSON Data Structure


In [ ]:
# Display full JSON structure for one algorithm as example
print("Example JSON structure for one algorithm:")
print("=" * 80)
if len(results_list) > 0:
    import json
    example = results_list[0]
    print(json.dumps(example, indent=2))
else:
    print("No results available")


## 12. Statistical Summary


In [ ]:
# Generate comprehensive statistical summary
print("=" * 80)
print("COMPREHENSIVE STATISTICAL SUMMARY")
print("=" * 80)

if baseline is not None:
    print(f"\nBaseline (ECDSA):")
    print(f"  File count: {baseline['file_count']:,}")
    print(f"  Total size: {baseline['total_size_gb']:.3f} GB")
    print(f"  Validation time: {baseline['validation_time_sec']:.2f} seconds ({baseline['validation_time_min']:.2f} minutes)")
    print(f"  Objects per second: {baseline.get('objects_per_second', 0):.2f}")

print(f"\nPost-Quantum Algorithms:")
for result in results_list:
    if result['algorithm'] != 'ecdsa-baseline':
        algo = result['algorithm']
        print(f"\n{algo.upper()}:")
        print(f"  File count: {result['file_count']:,}")
        print(f"  Total size: {result['total_size_gb']:.3f} GB")
        print(f"  Validation time: {result['validation_time_sec']:.2f} seconds ({result['validation_time_min']:.2f} minutes)")
        print(f"  Objects per second: {result.get('objects_per_second', 0):.2f}")
        
        # Size and time overhead
        if baseline is not None:
            size_oh = ((result['total_size_gb'] / baseline['total_size_gb'] - 1) * 100)
            time_oh = ((result['validation_time_sec'] / baseline['validation_time_sec'] - 1) * 100)
            print(f"  Size overhead: {size_oh:+.1f}%")
            print(f"  Time overhead: {time_oh:+.1f}%")
        
        # Signature verification details
        sig_ver = result.get('signature_verification', {})
        if sig_ver and isinstance(sig_ver, dict) and sig_ver.get('sampled', 0) > 0:
            print(f"  Signature verification:")
            print(f"    Sampled: {sig_ver.get('sampled', 0):,}")
            print(f"    Verified: {sig_ver.get('verified', 0):,}")
            print(f"    Failed: {sig_ver.get('failed', 0):,}")
            print(f"    ASN.1 errors: {sig_ver.get('asn1_extraction_failures', 0):,}")
            print(f"    Avg verify time: {sig_ver.get('avg_verify_time_ms', 0):.2f} ms")
            print(f"    Verification rate: {sig_ver.get('verification_rate_per_sec', 0):.1f} sig/s")
            print(f"    Signature size: {sig_ver.get('signature_size_avg_bytes', 0):.0f} bytes (expected: {sig_ver.get('expected_signature_size_bytes', 0)})")
        
        # File type breakdown
        ftb = result.get('file_type_breakdown', {})
        if isinstance(ftb, dict) and any(ftb.values()):
            print(f"  File types: {', '.join([f'{k}:{v:,}' for k, v in ftb.items() if v > 0])}")

print("\n" + "=" * 80)
